# K2ST DST TAHLİYE

## Step 1: Evacuation Process Started
- **State Number**: 2
- **State Code**: 10380002
- **State Name**: Evacuation Process Started
- **Description**: This state begins the evacuation process. A service call is made with the contract ID to fetch the required contract data. The request is matched with attributes like `ServicePointId` and `AccountId`. Based on the evaluation, the system will transition to either State 3 or State 6.


#### 1a. Çalışma Öncesi Script

In [ ]:
# Yok 

#### 1b. İstek Eşleştirme Scripti 

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping
def state_handler(machine, state, request_data, context, prev_state_response):
    # do business/mapping here
    return {"id":request_data["accountId"]}


#### 1c. State Context Değer Tutma Scripti

In [ ]:
# machine           : Currently executing machine instance
# state             : Currently executing state instance
# source            : Original trigger data
# context           : Machine stored context
# prevStateResponse : Previous executed state business logic response
# return
#     store dictionary  
def state_handler(machine, state, request_data, context, prev_state_response):
    store = {}
    
    attribute_codes = ['ServicePointId', 'AccountId']
    
    for code in attribute_codes:
        for attr in request_data['attributes']:
            if attr['attribute']['attributeCode'] == code:
                value = attr['value']
                if value is None and attr['values']:
                    value = attr['values'][0]['value']  # values içindeki value al
                store[code] = value
                break
        else:
            store[code] = None  # Eğer bulunamazsa None döner
    return store

#### 1d. Çalışma Sonrası Scripti

In [ ]:

# A service will be added here to check the historical data count of TermType. If this stage has been reached, it indicates that the number of Term Changes is less than 3.

# machine                 : Currently executing machine instance
# state                   : Currently executing state instance
# request_data            : Original trigger data
# context                 : Machine stored context
# prev_state_response     : Previous executed state business logic response
# current_state_response  : Currently executed state business logic response
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}

def state_handler(machine, state, request_data, context, prev_state_response, current_state_response):
    condition = {
        'ContractCancellationDate': 99991231235959
    }

    # Şimdi kalan kontrollerde sıra
    if all(next((attr['value'] if attr['value'] is not None else (attr['values'][0]['value'] if attr['values'] else None)
        for attr in request_data['attributes']
            if attr['attribute']['attributeCode'] == key), None) == value for key, value in condition.items()):
                return {"success": True, "goto": "10340003"}
    else:
        return {"success": True, "error": {"code": "END", "detail": "Terim değişikliği yapmaya uygun değilsiniz"}}


#### 1e. İş Mantığı

In [ ]:
"""md/search/{assetId}"""

'md/search/{assetId}'

## Step 2: Work Order Creation
- **State Number**: 3
- **State Code**: 10380003
- **State Name**: Work Order Creation
- **Description**: This state involves the creation of a work order for the evacuation process. The system waits for the response from the work order. If successful, it transitions to the next state. Otherwise, it might transition to a failure state based on the logic in place.

#### 2a. Çalışma Öncesi Scripti

In [ ]:
#Yok

#### 2b. İstek Eşleştirme Scripti 

In [ ]:
#Yok

#### 2c. State Context Değer Tutma Scripti

In [ ]:
#Yok

#### 2d. Çalışma Sonrası Scripti

In [ ]:
#Yok

## Step 3: Contract Terminated
- **State Number**: 3
- **State Code**: 10380004
- **State Name**: Contract Terminated
- **Description**: In this state, the contract is terminated, and relevant data such as the cancellation reason, request date, and cancellation date are updated. The system ensures that all necessary attributes are set before transitioning to the next state.


#### 3a. Çalışma Öncesi Scripti

In [ ]:
#Yok

#### 3b. İstek Eşleştirme Scripti 

In [ ]:
# machine              : Currently executing machine instance
# state                : Currently executing state instance
# request_data         : Original trigger data
# context              : Machine stored context
# prev_state_response  : Previous executed state business logic response
# return
#     json result of business/mapping
import json
from datetime import datetime

def state_handler(machine, state, request_data, context, prev_state_response, current_state_response) -> json:  
    
    current_date = datetime.now().strftime('%Y%m%d') + '000000'
    
    data = {
        "id": request_data["id"],
        "attributes": [
            {
                "attribute": {
                    "attributeCode": "ContractCancellationDate"
                },
                "value":current_date
            },
            {
                "attribute": {
                    "attributeCode": "ContractCancellationRequestDate"
                },
                "value": current_date
            },
            {
                "attribute": {
                    "attributeCode": "ContractCancellationReason"
                },
                "value": "SubscriberRequest"  
            },
            {
                "attribute": {
                    "attributeCode": "ContractNumber"
                },
                "value": next((attr['value'] for attr in request_data['attributes'] 
                               if attr['attribute']['attributeCode'] == "ContractNumber"), None)
            }
        ],
        "authorityGroup": request_data["authorityGroup"],
        "contractLockID": "string"  
    }
    
    return json.dumps(data, indent=4)


#### 3c. State Context Değer Tutma Scripti

In [ ]:
## Yok

#### 3d. Çalışma Sonrası Scripti

In [ ]:
# machine                 : Currently executing machine instance
# state                   : Currently executing state instance
# request_data            : Original trigger data
# context                 : Machine stored context
# prev_state_response     : Previous executed state business logic response
# current_state_response  : Currently executed state business logic response
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}
def state_handler(machine, state, request_data, context, prev_state_response, current_state_response):
    # do workflow here
    return {"success": True, "goto": "10380005"}


## Step 4: Accrual Calculation Generated
- **State Number**: 4
- **State Code**: 10380005
- **State Name**: Accrual Calculation Generated
- **Description**: A final accrual is generated for the terminated contract. The system calculates the closing accrual based on the contract's end date and field data (e.g., meter readings). It then moves to the next state.
- **Status**: Pending

#### 4a. Çalışma Öncesi Scripti

In [ ]:
# Yok

#### 4b. İstek Eşleştirme Scripti 

In [ ]:
#yok

#### 4c. State Context Değer Tutma Scripti

In [ ]:
#Yok

#### 4d. Çalışma Sonrası Scripti

In [ ]:
# machine                 : Currently executing machine instance
# state                   : Currently executing state instance
# request_data            : Original trigger data
# context                 : Machine stored context
# prev_state_response     : Previous executed state business logic response
# current_state_response  : Currently executed state business logic response
# return
#     {"success": True, "goto": "STATE_CODE"}
#     {"success": False, "error": {"code": "...", "detail": "..."}}
def state_handler(machine, state, request_data, context, prev_state_response, current_state_response):
    # do workflow here
    return {"success": True, "goto": "END"}


#### 4e. İş Mantığı

In [ ]:
"""http://thor-stage.edas1.com/host/v1//accruals/calculate"""

'http://thor-stage.edas1.com/host/v1//accruals/calculate'